**Import librarys**

In [2]:
from ultralytics import YOLO
from ultralytics import settings
import torch
import os
import cv2
from sklearn.metrics import classification_report

**Check torch version**

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))
print("Antal CPU-kärnor:", os.cpu_count())

**Check Ultralytics settings**

In [4]:
print(settings)

JSONDict("C:\Users\shool\AppData\Roaming\Ultralytics\settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "C:\\Users\\shool\\Dev\\Codenames-Board\\modelv12\\datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "0847b39e9ea20ed4a1701ad224a7b4090217127da71fb1cbe1040083b7cf976f",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": true,
  "vscode_msg": true
}


**Download Dataset**

In [5]:
from roboflow import Roboflow

# dotenv.load_dotenv()
# settings.reset()

# ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
# PROJECT_ID = os.getenv("PROJECT_OBJECT_DETECTION")
# VERSION = os.getenv("OBJECT_DETECTION_MODEL_VERSION")

# def download_dataset(PROJECT_ID, version, yolov):
#   rf = Roboflow(api_key=ROBOFLOW_API_KEY)
#   project = rf.workspace().project(PROJECT_ID)
# #   version = project.version(version)
#   dataset = project.version(version).download(yolov)
#   return dataset

# dataset = download_dataset(PROJECT_ID, 9, yolov="yolov11")



https://docs.ultralytics.com/modes/train/#train-settings

**Train model**

In [ ]:
def model_training(data):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO("yolo11n.pt")  # Laddar in förtränad modell
    model.to(device)
    results = model.train(
        data=data, 
        epochs=100, 
        imgsz=640, 
        batch=8,
        workers=os.cpu_count(),
        exist_ok=True
        )
    return results

results = model_training(data="modelv12.yaml")


**Validate model**

In [7]:
def model_validation():
    model = YOLO("runs/detect/train/weights/best.pt")
    metrics = model.val(
        data="modelv12.yaml", 
        imgsz=640, 
        batch=16, 
        conf=0.8, 
        iou=0.6, 
        device="0",
        dnn=True,
        exist_ok=True,
        save_json=True)
    return metrics


metrics = model_validation()
print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)
print(metrics.box.maps)


Ultralytics 8.3.21  Python-3.11.10 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\shool\Dev\Codenames-Board\modelv12\datasets\labels\val.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:14<00:00, 14.53s/it]


                   all         12        134          1          1      0.995      0.929
               class_0         12         69          1          1      0.995      0.979
               class_1         12         65          1          1      0.995      0.879
Speed: 2.7ms preprocess, 19.9ms inference, 0.0ms loss, 10.1ms postprocess per image
Saving runs\detect\val\predictions.json...
Results saved to runs\detect\val
0.9294441006932447
0.995
0.995
[     0.9795     0.87939]


Ja, detta är en slags utvärderingsrapport, men den skiljer sig från en klassisk "classification report" i termer av form och innehåll. I YOLO och andra objektidentifieringsmodeller rapporteras oftast precision, recall, mAP@0.5, och mAP@0.5:0.95 istället för den klassiska precision, recall och F1-score som används i en typisk klassificeringsrapport.

Här är en snabb översikt av vad raderna betyder:

- **Box(P):** Precision för objektidentifiering, som anger hur många av modellens positiva förutsägelser som var korrekta.
- **R:** Recall, eller hur många av de verkliga objekten som modellen korrekt identifierade.
- **mAP@0.5:** Mean Average Precision vid en tröskel på 0.5, vilket är ett mått på hur bra modellen är på att hitta och korrekt avgränsa objekten.
- **mAP@0.5:0.95:** mAP beräknat över flera trösklar (från 0.5 till 0.95) för att ge en mer generell bild av modellens prestanda.
Denna rapport är mer omfattande än en klassisk klassificeringsrapport och fokuserar på modellens prestanda för objektidentifiering snarare än ren klassificering.

**Prediction**

In [4]:
def predict_image(image_path, conf=0.8):

    model = YOLO("runs/detect/train/weights/best.pt")
    
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    results = model.predict(
        img, 
        conf=conf, 
        stream=True,
        save=True
    )
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = [int(coord.item()) for coord in box.xyxy[0]]
            label = int(box.cls.item())
            confidence = box.conf.item()
            
            # Rita bounding boxen på bilden
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"Label: {label}, Conf: {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


test_image = os.path.join(os.getcwd(), "test", "board_018.png")

predict_image(test_image)





0: 416x640 25 class_0s, 25 class_1s, 21.7ms
Speed: 4.5ms preprocess, 21.7ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict
